In [5]:
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl

# After running this, paste your Hugging Face Access Token when prompted
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `LLM` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credentia

In [8]:
!pip install -U bitsandbytes

In [ ]:
# import torch
# from datasets import load_dataset
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     BitsAndBytesConfig,
#     TrainingArguments,
# )
# from peft import LoraConfig
# from trl import SFTTrainer

# # 1. Configuration
# # ------------------------------------------------------------------------------------
# # The model we want to fine-tune
# base_model = "meta-llama/Llama-3.2-3B-Instruct"

# # The dataset we created
# dataset_name = "tahamajs/bitcoin-llm-finetuning-dataset"
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# # pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct")
# # # The name of our new, fine-tuned model
# # # ❗️ IMPORTANT: Replace "your-username" with your actual Hugging Face username
# # new_model = "your-username/llama-3.2-8b-instruct-bitcoin-analyst"

# 2. Load the Dataset
# ------------------------------------------------------------------------------------
dataset = load_dataset(dataset_name, split="train")

# # 3. Configure QLoRA (4-bit Quantization and LoRA)
# # ------------------------------------------------------------------------------------
# # 4-bit quantization configuration
# # From Cell 3 of the notebook

# import torch
# from transformers import BitsAndBytesConfig

# # 4-bit quantization configuration
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,                 # This is the master switch to enable 4-bit loading
#     bnb_4bit_quant_type="nf4",         # A high-quality quantization format
#     bnb_4bit_compute_dtype=torch.bfloat16, # The data type used for calculations
# )

# peft_config = LoraConfig(
#     lora_alpha=16,
#     lora_dropout=0.1,
#     r=16,  # The rank of the adapters, a key parameter for LoRA
#     bias="none",
#     task_type="CAUSAL_LM",
#     # This tells the script which parts of the Llama model to attach adapters to
#     target_modules=[
#         "q_proj",
#         "k_proj",
#         "v_proj",
#         "o_proj",
#         "gate_proj",
#         "up_proj",
#         "down_proj",
#     ],
# )
# # 4. Load the Base Model and Tokenizer
# # ------------------------------------------------------------------------------------
# # Load the model with 4-bit quantization
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config,
#     device_map="auto" # Automatically place the model on the available GPU
# )
# model.config.use_cache = False
# model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Llama models do not have a pad token, so we use the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# --- 3. Format the Dataset ---
# This function creates a new 'text' column with the fully formatted prompt.
def format_prompts_for_sft(example):
    messages = [
        {"role": "user", "content": f"{example['instruction']}\n{example['input']}"},
        {"role": "assistant", "content": example['output']}
    ]
    # The tokenizer's chat template handles all the special tokens
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

print("Formatting dataset into Llama 3.2 chat template...")
# Create the new dataset with the 'text' column
formatted_dataset = dataset.map(format_prompts_for_sft)
print("✅ Dataset formatting complete.")

# 5. Set up the Training Arguments and Trainer
# ------------------------------------------------------------------------------------
# Training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True, # Use bfloat16 for better performance on modern GPUs
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Create the SFTTrainer
# The SFTTrainer from TRL simplifies the supervised fine-tuning process.
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    # Here we format the dataset into the Llama 3.2 chat template
    # dataset_text_field="text", # The SFTTrainer expects a single text field. We will create it on the fly.
    # max_seq_length=20480,
    # tokenizer=tokenizer,
    args=training_arguments,
    # packing=False,
)

# 6. Train the Model
# ------------------------------------------------------------------------------------
print("🚀 Starting the fine-tuning process...")
trainer.train()
print("✅ Training complete!")

# 7. Save the Fine-Tuned Model
# ------------------------------------------------------------------------------------
# This saves the LoRA adapter, not the full model
print(f"💾 Saving model to {new_model}...")
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)
print("✅ Model saved successfully.")

Formatting dataset into Llama 3.2 chat template...


Map:   0%|          | 0/2312 [00:00<?, ? examples/s]

✅ Dataset formatting complete.


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/2312 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2312 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2312 [00:00<?, ? examples/s]

🚀 Starting the fine-tuning process...


Step,Training Loss


In [10]:
pip install -U bitsandbytes